In [12]:
import pandas as pd

In [57]:
from modules.util import load_json, load_pkl
from modules.train import clean_text, word_to_num, seq_padding, SentimentAnalysisRNN, calculate_accuracy

reviews = load_json("./database/venom_review.json")

In [26]:
target_data = {}
target_data["sentiment"] = []
target_data["review"] = []
for i,review in enumerate(reviews["reviews"]):
    target_data["sentiment"].append(int(review["isFresh"]))
    target_data["review"].append(review["quote"])

target_data = pd.DataFrame(target_data)

In [36]:
from nltk.tokenize import TreebankWordTokenizer
word2idx = load_pkl("./word2index.pkl")

# 텍스트 데이터 전처리
target_data = target_data.drop_duplicates(subset="review")
target_data["review_cleaned"] = target_data["review"].apply(clean_text)

# Tokenization
tokenizer = TreebankWordTokenizer()
target_data.loc[:, "review_tokenized"] = target_data.loc[:, "review_cleaned"].apply(tokenizer.tokenize)
target_data["review_numbered"] = target_data["review_tokenized"].apply(lambda _X: [word_to_num(word,word2idx) for word in _X])
target_data["token_length"] = target_data["review_numbered"].apply(lambda _X: len(_X))

max(target_data["token_length"])

27

In [ ]:
import torch

model = SentimentAnalysisRNN(68505, 256, 1024, 2)
model.load_state_dict(torch.load("./best_model_checkpoint-2024-11-04 03:19:52.630041.pth", weights_only=True))

SentimentAnalysisRNN(
  (embed): Embedding(68505, 256)
  (rnn): RNN(256, 1024, batch_first=True)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [73]:
target = seq_padding(target_data["review_numbered"])
label = torch.tensor(target_data["sentiment"].to_numpy())

In [ ]:
model.eval()
with torch.no_grad():
    total = target.shape[0]
    correct = (torch.argmax(model(target), dim=1) == label).sum()
    print(correct / total)

tensor(0.5484)
